## CNN digit classifier (MNIST-as-JPG)

Train a small Keras CNN to classify digits **0–9** from grayscale JPGs on disk.

Expected layout:

```
DATA_DIR/0/*.jpg
DATA_DIR/1/*.jpg
...
DATA_DIR/9/*.jpg
```


In [ ]:
import os

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as pltimg

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout

## Quick image preview

Optional sanity check: display 3 sample images and print their shapes (paths are hard-coded; adjust if needed).

In [ ]:
sample_paths = [
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/8/img_14050.jpg",
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/4/img_10034.jpg",
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/3/img_57.jpg",
]

sample_imgs = [pltimg.imread(p) for p in sample_paths]

fig, axes = plt.subplots(1, len(sample_imgs), figsize=(3 * len(sample_imgs), 3))
if len(sample_imgs) == 1:
    axes = [axes]

for i, (ax, img) in enumerate(zip(axes, sample_imgs), start=1):
    ax.imshow(img, cmap="gray")
    ax.axis("off")
    print(f"sample_img_{i}.shape:", img.shape)

plt.tight_layout()
plt.show()

## Data pipeline (tf.data)

`tf.data` pipeline: decode grayscale JPG → resize to `IMG_SIZE` → rescale to **[0,1]** → one-hot labels.

Datasets:
- `train_ds`: shuffled + augmented
- `val_ds`: no augmentation
- `test_ds`: no augmentation, no shuffle (held-out evaluation)

Uses `cache()` + `prefetch()` for speed.


In [ ]:
SEED = 42
IMG_SIZE = (28, 28)
BATCH_SIZE = 32
SHUFFLE_BUFFER = 1000
AUTOTUNE = tf.data.AUTOTUNE
DATA_DIR = "/kaggle/input/mnistasjpg/trainingSet/trainingSet"
MODEL_PATH = "model2.keras"

tf.keras.utils.set_random_seed(SEED)

augmentation_layer = tf.keras.Sequential(
    [
        tf.keras.layers.RandomRotation(10 / 360.0),
        tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.RandomZoom(0.1),
    ]
)

rescale_layer = tf.keras.layers.Rescaling(1.0 / 255.0)

def augment(images, labels):
    images = augmentation_layer(images, training=True)
    return images, labels

## Train/val/test split

Stratified **70/15/15** split per digit folder (`0..9`) under `DATA_DIR`, then shuffle within each split.


In [ ]:
from pathlib import Path

# Stratified 70/15/15 split per digit folder
train_paths, train_labels = [], []
val_paths, val_labels = [], []
test_paths, test_labels = [], []

for d in range(10):
    cls_paths = sorted(str(p) for p in (Path(DATA_DIR) / str(d)).glob("*.jpg"))
    rng = np.random.default_rng(SEED + d)
    rng.shuffle(cls_paths)

    n = len(cls_paths)
    n_train = int(0.70 * n)
    n_val = int(0.15 * n)
    # remainder goes to test to ensure totals match exactly
    n_test = n - n_train - n_val

    train_paths += cls_paths[:n_train]
    train_labels += [d] * n_train

    val_start = n_train
    val_end = n_train + n_val
    val_paths += cls_paths[val_start:val_end]
    val_labels += [d] * n_val

    test_paths += cls_paths[val_end:]
    test_labels += [d] * n_test

In [ ]:
# Shuffle within each split
split_rng = np.random.default_rng(SEED)
train_perm = split_rng.permutation(len(train_paths))
val_perm = split_rng.permutation(len(val_paths))
test_perm = split_rng.permutation(len(test_paths))

train_paths = [train_paths[i] for i in train_perm]
train_labels = [train_labels[i] for i in train_perm]
val_paths = [val_paths[i] for i in val_perm]
val_labels = [val_labels[i] for i in val_perm]
test_paths = [test_paths[i] for i in test_perm]
test_labels = [test_labels[i] for i in test_perm]

print("Dataset split:")
print(f"- total: {len(train_paths) + len(val_paths) + len(test_paths)}")
print(f"- train: {len(train_paths)}")
print(f"- val:   {len(val_paths)}")
print(f"- test:  {len(test_paths)}")

In [ ]:
NUM_CLASSES = 10

def load_example(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.resize(img, IMG_SIZE)
    img = rescale_layer(tf.cast(img, tf.float32))
    return img, tf.one_hot(label, depth=NUM_CLASSES)

In [ ]:
train_ds = (
    tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
    .map(load_example, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .shuffle(SHUFFLE_BUFFER, seed=SEED, reshuffle_each_iteration=True)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .prefetch(AUTOTUNE)
)

val_ds = (
    tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
    .map(load_example, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTOTUNE)
)

test_ds = (
    tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
    .map(load_example, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTOTUNE)
)

## Split sanity check

Verify the three splits are **disjoint** and roughly **70/15/15** per class.


In [ ]:
from collections import Counter

# Sanity check (part 1): disjointness
train_set, val_set, test_set = set(train_paths), set(val_paths), set(test_paths)

overlap_train_val = train_set & val_set
overlap_train_test = train_set & test_set
overlap_val_test = val_set & test_set

print(f"train: {len(train_paths)}")
print(f"val:   {len(val_paths)}")
print(f"test:  {len(test_paths)}")
print(f"overlap (train/val):  {len(overlap_train_val)}")
print(f"overlap (train/test): {len(overlap_train_test)}")
print(f"overlap (val/test):   {len(overlap_val_test)}")



In [ ]:
# Sanity check (part 2): per-class counts + overlap examples
train_counts = Counter(int(Path(p).parent.name) for p in train_paths)
val_counts = Counter(int(Path(p).parent.name) for p in val_paths)
test_counts = Counter(int(Path(p).parent.name) for p in test_paths)

print("per-class (train/val/test):")
for d in range(10):
    print(f"- {d}: {train_counts[d]}/{val_counts[d]}/{test_counts[d]}")

def _print_overlap_examples(name, overlap_paths, limit=10):
    if overlap_paths:
        print(f"examples ({name}):")
        for p in sorted(overlap_paths)[:limit]:
            print(f"- {p}")

_print_overlap_examples("train/val", overlap_train_val)
_print_overlap_examples("train/test", overlap_train_test)
_print_overlap_examples("val/test", overlap_val_test)



## Model

A small CNN that learns to map each 28×28 grayscale image to one of the 10 digit classes (0–9).

Trained with Adam and categorical cross-entropy; learning rate is reduced automatically if validation loss stops improving.


In [ ]:
model = Sequential(
    [
        Input(shape=(28, 28, 1)),
        Conv2D(32, (3, 3), activation="relu", padding="same"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation="relu", padding="same"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.3),
        Dense(10, activation="softmax"),
    ]
)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(MODEL_PATH, monitor="val_accuracy", mode="max", save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5),
]

history = model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=callbacks)

## Training curve

Train vs validation **accuracy** over epochs; a growing gap usually means **overfitting**.


In [ ]:
# Plot training vs validation accuracy
acc = history.history.get("accuracy")
val_acc = history.history.get("val_accuracy")

if acc is None or val_acc is None:
    raise KeyError(
        "Missing accuracy history. Available keys: " + ", ".join(sorted(history.history.keys()))
    )

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(7, 4))
plt.plot(epochs, acc, label="train accuracy")
plt.plot(epochs, val_acc, label="val accuracy")
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

## Test evaluation (held-out test set)

Evaluate the best saved model on the held-out `test_ds` (no augmentation, no shuffle). Optionally plot a confusion matrix.


In [ ]:
best_model = load_model(MODEL_PATH)

test_loss, test_acc = best_model.evaluate(test_ds, verbose=1)
print(f"test loss: {test_loss:.4f}")
print(f"test acc:  {test_acc:.4f}")

In [ ]:
# confusion matrix
all_true = []
all_pred = []
for images, labels in test_ds:
    y_true = tf.argmax(labels, axis=1)
    y_pred = tf.argmax(best_model(images, training=False), axis=1)
    all_true.append(y_true)
    all_pred.append(y_pred)

all_true = tf.concat(all_true, axis=0)
all_pred = tf.concat(all_pred, axis=0)

cm = tf.math.confusion_matrix(all_true, all_pred, num_classes=NUM_CLASSES).numpy()

plt.figure(figsize=(7, 6))
plt.imshow(cm, interpolation="nearest", cmap="Blues")
plt.title("Confusion matrix (test set)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()
plt.xticks(range(NUM_CLASSES))
plt.yticks(range(NUM_CLASSES))

# annotate cells (small enough for 10 classes)
for i in range(NUM_CLASSES):
    for j in range(NUM_CLASSES):
        plt.text(j, i, str(cm[i, j]), ha="center", va="center", fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
sample_paths = [
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/2/img_7169.jpg",
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/6/img_16973.jpg",
    "/kaggle/input/mnistasjpg/trainingSet/trainingSet/7/img_23407.jpg",
]

fig, axes = plt.subplots(1, len(sample_paths), figsize=(3 * len(sample_paths), 3))
if len(sample_paths) == 1:
    axes = [axes]

for ax, path in zip(axes, sample_paths):
    pil_img = tf.keras.utils.load_img(path, color_mode="grayscale")
    img_array = tf.keras.utils.img_to_array(pil_img)

    input_batch = rescale_layer(tf.expand_dims(img_array, axis=0))
    pred_probs = best_model.predict(input_batch, verbose=0)
    pred_class = int(tf.argmax(pred_probs, axis=1)[0])

    ax.imshow(img_array.squeeze(), cmap="gray")
    ax.set_title(f"pred: {pred_class}")
    ax.axis("off")

plt.tight_layout()
plt.show()